In [1]:
import pandas as pd
from src.oil.futures import fetch_forward_curve_from_contracts

In [2]:
fetch_forward_curve_from_contracts()

TypeError: fetch_forward_curve_from_contracts() missing 1 required positional argument: 'symbols'

In [70]:
import src.secrets as secrets
import requests
import pandas as pd
base_url = "https://api.eia.gov/v2/petroleum/pnp/inpt2/data/"
res = requests.get(base_url, params={"api_key": secrets.EIA_KEY, "frequency": "monthly", "data[0]": "value", "sort[0][column]": "period", "sort[0][direction]": "desc", "offset": 0, "length": 5000})
data = res.json()
df = pd.DataFrame(data["response"]["data"])

In [2]:
import streamlit as st

In [3]:
st.secrets["EIA_KEY"]

'dRh3IDfmi3YRYvXSgczY0q8mZsLzoOd6TUyOgb8k'

In [68]:
df = pd.DataFrame(data["response"]["data"])
# Convert period to datetime
df["Date"] = pd.to_datetime(df["period"])
df["value"] = pd.to_numeric(df["value"], errors="coerce")

# Separate state-level rows
df_states = df[df["duoarea"].str.startswith("S")].copy()

# Separate main PADD-level rows
valid_padds = ["R10", "R20", "R30", "R40", "R50"]
df_padds = df[df["duoarea"].isin(valid_padds)].copy()




df_top_pivot = df_padds.pivot_table(index="Date", columns="duoarea", values="value", aggfunc="sum")
df_top_pivot = df_top_pivot.rename(columns={
    "R10": "PADD 1",
    "R20": "PADD 2",
    "R30": "PADD 3",
    "R40": "PADD 4",
    "R50": "PADD 5"
})

df_top_pivot["US Total"] = df_top_pivot["PADD 1"] + df_top_pivot["PADD 2"] + df_top_pivot["PADD 3"] + df_top_pivot["PADD 4"] + df_top_pivot["PADD 5"]

In [69]:
df_top_pivot

duoarea,PADD 1,PADD 2,PADD 3,PADD 4,PADD 5,US Total
Date,,,,,,
2020-03-01,2274,69631,279585,30696,27292,409478
2020-04-01,2017,57674,254764,27470,27320,369245
2020-05-01,1945,42072,217601,23722,25508,310848
2020-06-01,2051,48008,224943,25009,23816,323827
2020-07-01,2070,55165,241009,26964,27045,352253
...,...,...,...,...,...,...
2024-11-01,1513,57440,298912,33679,22370,413914
2024-12-01,1458,58703,312980,34033,22838,430012
2025-01-01,1480,57717,305801,32581,22918,420497
